In [1]:
import pandas as pd
import duckdb 

df = pd.read_excel(
    "../data/resultat-ansokningsomgang-2024.xlsx", sheet_name="Tabell 3", skiprows=5
)
df.head()

,Utbildningsområde,SUN5 inriktning,SUN5 inriktning namn,Utbildningsnamn,Beslut,Diarienummer,Flera kommuner,Antal kommuner,Län,Kommun,...,Sökta utbildningsomgångar,Beviljade utbildningsomgångar,Sökta platser per utbildningsomgång,Sökta platser totalt,Beviljade platser utbildningsomgång 1,Beviljade platser utbildningsomgång 2,Beviljade platser utbildningsomgång 3,Beviljade platser utbildningsomgång 4,Beviljade platser utbildningsomgång 5,Beviljade platser totalt
0,Data/IT,481ab,"Utbildningar till programmerare, spel",AI Programmer,Avslag,MYH 2024/3742,Nej,1,Västerbotten,Umeå,...,3,0,30,90,0,0,0,0,0,0
1,Data/IT,481ab,"Utbildningar till programmerare, spel","Forsbergs, Spelutvecklare – Game Programming",Avslag,MYH 2024/4174,Nej,1,Stockholm,Stockholm,...,3,0,20,60,0,0,0,0,0,0
2,Data/IT,481ab,"Utbildningar till programmerare, spel",Futuregames Game Programmer Specialization,Avslag,MYH 2024/3029,Ja,2,Flera kommuner,Flera kommuner,...,3,0,30,90,0,0,0,0,0,0
3,Data/IT,481ab,"Utbildningar till programmerare, spel",Game Programmer,Avslag,MYH 2024/3425,Nej,1,Blekinge,Karlshamn,...,3,0,25,75,0,0,0,0,0,0
4,Data/IT,481ab,"Utbildningar till programmerare, spel",Spelprogrammerare,Beviljad,MYH 2024/3852,Nej,1,Skåne,Malmö,...,4,3,35,140,35,35,35,0,0,105


In [2]:
df_regions = duckdb.query(
    """
    SELECT 
        län, 
        COUNT_IF(beslut ='Beviljad') AS Beviljade
    FROM df 
    WHERE län != 'Flera kommuner'
    GROUP BY
        län
    ORDER BY 
        beviljade 
    DESC
"""
).df()

df_regions

,Län,Beviljade
0,Stockholm,82.0
1,Västra Götaland,63.0
2,Skåne,37.0
3,Östergötland,16.0
4,Jönköping,13.0
5,Örebro,10.0
6,Västernorrland,6.0
7,Kalmar,5.0
8,Kronoberg,5.0
9,Västmanland,5.0


In [3]:
import json

with open("../assets/swedish_regions.geojson", "r") as fp:
    json_data = json.load(fp)

json_data.get("features")[3].get("properties")

{'admin_level': '4',
 'boundary': 'administrative',
 'name': 'Jämtlands län',
 'ref:se:län:bokstav': 'Z',
 'ref:se:länskod': '23',
 'type': 'multipolygon'}

In [ ]:
def sum_field_by_filter(df, filter_field, filter_value, sum_field):
    """
    Filtrerar df på filter_field == filter_value och summerar värden i sum_field.
    
    :param df: Pandas DataFrame
    :param filter_field: Kolumnnamn att filtrera på, t.ex. 'Utbildningsområde'
    :param filter_value: Värde att filtrera efter, t.ex. 'Data/IT'
    :param sum_field: Kolumn att summera, t.ex. 'Antal beviljade platser'
    :return: Summan av sum_field efter filtrering
    """
    filtered = df.query(f"{filter_field} == @filter_value")
    return filtered[sum_field].sum()

# 🧪 Exempel på användning:
total = sum_field_by_filter(df, "Utbildningsområde", "Data/IT", "Antal beviljade platser")
print(f"Total summa för Data/IT: {total}")
